## **Install The Library And Load Data From the Cloud**

In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Load the dataset from Google Drive
df_ratings = pd.read_csv('/content/drive/My Drive/rating.csv')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
rating = pd.read_csv('/content/drive/My Drive/rating.csv', usecols=['userId','movieId','rating'], dtype={'userId': 'Int32', 'movieId': 'int32', 'rating': 'float32'})
rating.head()


In [ ]:
movies =  pd.read_csv('movie.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})


## Data merging



In [ ]:
all_in_one = pd.merge(rating,movies,on='movieId')
all_in_one.head()

## Rating Calculation

In [ ]:
all_in_one = pd.merge(rating,movies,on='movieId')
all_in_one.head()
Calculate_Rating = all_in_one.dropna(axis = 0, subset = ['title'])
movie_ratingCount = (Calculate_Rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
movie_ratingCount

## Data Combination

In [ ]:
combine_all = Calculate_Rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
combine_all.head()

## Feltring Movies With Nombre of Rating Less Than 1000

In [ ]:
minimum_number_of_ratings = 1000
rating_popular_movie= combine_all.query('totalRatingCount >= @minimum_number_of_ratings')
rating_popular_movie.head()
rating_popular_movie.shape

## Creation Pivot Creation

In [ ]:
matrix_pivot=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)

In [ ]:
matrix_pivot.head()

## Sparse Matrix

In [ ]:
movie_features_df_matrix = csr_matrix(matrix_pivot.values)

## Nearest Neighbors Algorithme (KNN)

In [ ]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

## Choose The Movie

In [ ]:
# Input the title of the movie you want to use as the query movie
# Get the index of the query movie
query_movie_title = input("Enter the title of the movie you want to use as the query: ")


## Movie Recommendations

In [ ]:
query_index = matrix_pivot.index.get_loc(query_movie_title)
# Use the trained kNN model to find the nearest neighbors to the query movie
distances, indices = model_knn.kneighbors(matrix_pivot.iloc[query_index, :].values.reshape(1, -1), n_neighbors=10)
# Print the recommended movies along with their distances from the query movie
print("Recommendations for {0}:\n".format(matrix_pivot.index[query_index]))
for i in range(0, len(distances.flatten())):
    if i == 0:
        print("Original movie: {0}".format(matrix_pivot.index[query_index]))
    else:
        print("{0}: {1}, with distance of {2}".format(i, matrix_pivot.index[indices.flatten()[i]], distances.flatten()[i]))